# Background 

`critbit.move` structures are as follows (comments removed for brevity):

```rust

struct CritBitTree<V> has store {
    root: u64,
    inner_nodes: vector<InnerNode>,
    outer_nodes: vector<OuterNode<V>>
}

struct InnerNode has store {
    critical_bit: u8,
    parent_index: u64,
    left_child_index: u64,
    right_child_index: u64
}

struct OuterNode<V> has store {
    key: u128,
    value: V,
    parent_index: u64,
}
```

`critqueue.benchmark` uses an `address` for `V`, yielding the following byte sizes:

In [ ]:
inner_node_size_bytes = 1 + 8 + 8 + 8
inner_node_size_bytes

In [ ]:
outer_node_size_bytes = 16 + 32 + 8 # 32-byte address.
outer_node_size_bytes

With the exception of the first outer node insertion at the root, each insertion to the tree adds both an inner node and an outer node. Hence in the general case of `n` outer nodes there are `n - 1` outer nodes, such that the size of the tree is approximated by `n * (s_i + s_o)`, or the number of inserted keys times the combined size of an inner and outer node:

In [ ]:
bytes_per_key = inner_node_size_bytes + outer_node_size_bytes
bytes_per_key

Per the [storage gas optimization principles](https://aptos.dev/concepts/base-gas/#storage-gas-1), per-byte writes cost 0.02 octals and per-item writes cost 40 octals:

In [ ]:
gas_per_written_byte = 0.02
gas_per_written_byte

In [ ]:
gas_per_written_item = 40
gas_per_written_item

And for reads:

In [ ]:
gas_per_read_byte = 0.004
gas_per_read_byte

In [ ]:
gas_per_read_item = 8
gas_per_read_item

As defined in [`instruction.rs`](https://github.com/aptos-labs/aptos-core/blob/main/aptos-move/aptos-gas/src/transaction.rs), the minimum transaction gas is 1,500,000 units, or 150 octals with a scale factor of 10,000:

In [ ]:
min_gas = 1_500_000 / 10_000
min_gas

With all data contained in a single item in global storage, read and write operation costs can thus be approximated by the number of keys in the tree:

In [ ]:
def expected_read_cost(n_keys) -> int:
    # Borrow `TreeStore`, borrow `CritBitTree`,
    # then read the bytes within.
    return int(min_gas + \
    + gas_per_read_item \
    + gas_per_read_item \
    + n_keys * bytes_per_key * gas_per_read_byte)

In [ ]:
expected_read_cost(1)

In [ ]:
expected_read_cost(10)

In [ ]:
expected_read_cost(100)

In [ ]:
expected_read_cost(1000)

In [ ]:
def expected_write_cost(n_keys) -> int:
    # Borrow `TreeStore`, borrow `CritBitTree`,
    # then read the bytes within.
    return int(min_gas + \
    + gas_per_read_item \
    + gas_per_written_item \
    + n_keys * bytes_per_key * gas_per_written_byte)

In [ ]:
expected_write_cost(1)

In [ ]:
expected_write_cost(10)

In [ ]:
expected_write_cost(100)

In [ ]:
expected_write_cost(1000)

# Imports

In [ ]:
import pandas as pd
import time

from aptos_sdk.account import Account
from aptos_sdk.bcs import Serializer
from aptos_sdk.client import RestClient, FaucetClient
from aptos_sdk.transactions import (
    EntryFunction,
    ModuleId,
    TransactionArgument as TxArg,
    TransactionPayload as TxPayload)
from aptos_sdk.type_tag import StructTag
from IPython.display import clear_output
from typing import Any, Dict
from pathlib import Path

# Account via keyfile

In [ ]:
# Relative path to hot keyfile for testing only.
# File should contain hex key only.`
key_file = sorted(Path('../../.secrets').glob('*.key'))[0]
with open(key_file) as f: # Open file.
    key = f.readline().rstrip() # Get key.
# Get account from key.
account = Account.load_key(key)
# Print account address in hex.
account.address().hex()

# REST Client 

In [ ]:
# Declare base URL for devnet REST API.
client_url = 'https://fullnode.devnet.aptoslabs.com/v1'
# Declare base URL for faucet.
faucet_url = 'https://faucet.devnet.aptoslabs.com'
# Initialize client.
client = RestClient(client_url)
# Initialize faucet.
faucet = FaucetClient(faucet_url, client)
# Get account balance
client.account_balance(account.address())

In [ ]:
# Fund from the faucet
faucet.fund_account(account.address().hex(), 1000000000)

In [ ]:
# Get account balance
client.account_balance(account.address())

In [ ]:
def execute(function: str, args: list) -> str:
    """Call entry function with args, returning tx hash
    
    args should be of format
    [[4, Serializer.u64], [2, Serializer.u128]]
    """
    # Construct entry function payload.
    payload = EntryFunction.natural(
        str(module_id),
        function,
        [],
        [TxArg(a[0], a[1]) for a in args])
    # Generate a signed transaction from the payload.
    signed_tx = client.create_single_signer_bcs_transaction(
        account, TxPayload(payload))
    # Submit signed transaction, returning transaction ID.
    return client.submit_bcs_transaction(signed_tx)

In [ ]:
def get_tx_json(hash: str) -> Dict[str, Any]:
    """Query a transaction by hash, returning JSON data"""
    while(True):
        response = client.client.get(
            f'{client.base_url}/transactions/by_hash/{hash}')
        # Assert successful response.
        assert response.status_code == 200, hash
        # If transaction has cleared as user tx:
        if response.json()['type'] == 'user_transaction':
            # Return its JSON data.
            return response.json()
        # Otherwise try again, after waiting.
        time.sleep(0.5) # Else you might get rate-limited.

In [ ]:
def print_tx_url(version: str):
    """Print URL to transaction view on explorer"""
    explorer = 'https://aptos-explorer.netlify.app'
    print(f'{explorer}/txn/{version}')

In [ ]:
def tx_diagnostics(tx_json: Dict[str, Any]):
    """Print gas used or link to failed tx"""
    if tx_json['success'] == True:
        print(f'Gas used: {tx_json["gas_used"]}')
    else:
        print_tx_url(tx_json['version'])

# Critbit interface

In [ ]:
# Declare module name.
module = 'critbit_benchmark'
# Get module ID.
module_id = ModuleId(account.address(), module)
# Get tree store struct tag.
struct_tag = StructTag(account.address(),
                       module,
                      'TreeStore',
                       [])
# Check that account has one.
client.account_resource(account.address(),
                        struct_tag.__str__())

In [ ]:
def insert(key: int) -> str:
    """Insert given key, returning tx ID"""
    return execute('insert', [
        [key, Serializer.u128],
    ])

In [ ]:
def pop(key: int) -> str:
    """Pop given key, returning tx ID"""
    return execute('pop', [
        [key, Serializer.u128],
    ])

In [ ]:
def pop_twice(key_1: int, key_2: int) -> str:
    """Pop given keys, returning tx ID"""
    return execute('pop_twice', [
        [key_1, Serializer.u128],
        [key_2, Serializer.u128],
    ])

In [ ]:
def borrow(key: int) -> str:
    """Pop given key, returning tx ID"""
    return execute('borrow', [
        [key, Serializer.u128],
    ])

In [ ]:
def clear() -> str:
    """Clear `CritBitTree`, returning tx ID"""
    return execute('clear', [])

In [ ]:
def reset() -> str:
    """Reset `TreeStore`, returning tx ID"""
    return execute('reset', [])

In [ ]:
# Reset.
tx_diagnostics(get_tx_json(reset()))

# Simple operations 

Insert a single key:

In [ ]:
expected_write_cost(1)

In [ ]:
tx_diagnostics(get_tx_json(insert(1)))

Immutably borrow the single key:

In [ ]:
expected_read_cost(1)

In [ ]:
tx_diagnostics(get_tx_json(borrow(1)))

Pop the key:

In [ ]:
expected_write_cost(1)

In [ ]:
tx_diagnostics(get_tx_json(pop(1)))

# Unbalanced insertion

Insert keys in the following prder, creating an unbalanced tree of maximum height:
1. `0b0`
2. `0b10`
3. `0b100`

...

128. `0b10000...`

Here, each successive insertion has to walk more nodes during a search, producing the following tree:

>                   127th
>                  /     \
>               126th    100000000000000.....
>             ...
>            2nd
>           /   \
>         1st   100 
>        /   \
>      0th   10
>     /   \
>     0   1

In particular, by inserting smaller keys first, the insertion algorithm has to walk all the way down then all the way back up the tree.

In [ ]:
# Reset tree store, waiting until tx clears.
tx_diagnostics(get_tx_json(reset()))
# Init log for bit number, expected
# gas, and actual gas used.
gas = []
for i in range (128): # Loop over all bits:
    key = 0 # Assume no bitshift.
    if i != 0: # If should shift.
        key = 1 << i # Shift accordingly.
    # Store JSON for key insertion tx.
    tx_json = get_tx_json(insert(key))
    # Assert successful tx.
    assert tx_json['success'] == True
    # Log expected and actual gas for shift.
    insert_gas = {
        'height': i,
        'gas_expected': expected_write_cost(i + 1),
        'gas_actual': int(tx_json['gas_used'])
    }
    gas.append(insert_gas) # Append to gas log.
    # Print gas used.
    print(f'Post-insertion height: {insert_gas["height"]}, '
          f'expected gas: {insert_gas["gas_expected"]}, '
          f'actual gas: {insert_gas["gas_actual"]}')
    clear_output(wait=True) # Clear terminal output.

Plot results, noting that expected gas costs do not account for instruction costs associated with traversal:

In [ ]:
df = pd.DataFrame(gas)

In [ ]:
ax = df.plot(x='height',
             y='gas_expected')
df.plot(x='height',
        y='gas_actual',
        grid=True,
        ax=ax);
ax.set_ylim(ymin=0)

Repeat, but insert at the bottom of the tree instead of the top:

In [ ]:
# Reset tree store, waiting until tx clears.
tx_diagnostics(get_tx_json(reset()))
# Init log for bit number, expected
# gas, and actual gas used.
gas = []
n_bits = 128 # Number of bits in a key.
for i in range (n_bits): # Loop over all bits:
    key = 0 # Assume no bitshift.
    if i != n_bits - 1: # If should shift.
         # Shift accordingly.
        key = 1 << (n_bits - 1 - i)
    # Store JSON for key insertion tx.
    tx_json = get_tx_json(insert(key))
    # Assert successful tx.
    assert tx_json['success'] == True
    # Log expected and actual gas for shift.
    insert_gas = {
        'height': i,
        'gas_expected': expected_write_cost(i + 1),
        'gas_actual': int(tx_json['gas_used'])
    }
    gas.append(insert_gas) # Append to gas log.
    # Print gas used.
    print(f'Post-insertion height: {insert_gas["height"]}, '
          f'expected gas: {insert_gas["gas_expected"]}, '
          f'actual gas: {insert_gas["gas_actual"]}')
    clear_output(wait=True) # Clear terminal output.

Now traversal instructions are effectively cut in half, but estimates are still off:

In [ ]:
df2 = pd.DataFrame(gas)
ax2 = df2.plot(x='height',
               y='gas_expected')
df2.plot(x='height',
         y='gas_actual',
         grid=True,
         ax=ax2);
ax2.set_ylim(ymin=0);

# Balanced insertion, lookup, removal

For a balanced tree, instead insert the values:
* `0b00000000`
* `0b00000001`
* `0b00000010`
* `0b00000011`

...

* `0b11111111`

In [ ]:
# Reset tree store, waiting until tx clears.
tx_diagnostics(get_tx_json(reset()))
# Init log for bit number, expected
# gas, and actual gas used.
gas = []
for i in range (128): # Loop over all bits:
    # Store JSON for key insertion tx.
    tx_json = get_tx_json(insert(i))
    # Assert successful tx.
    assert tx_json['success'] == True
    # Log expected and actual gas for shift.
    insert_gas = {
        'n_keys': i,
        'gas_expected': expected_write_cost(i + 1),
        'gas_actual': int(tx_json['gas_used'])
    }
    gas.append(insert_gas) # Append to gas log.
    # Print gas used.
    print(f'Pre-insertion leaf count: {insert_gas["n_keys"]}, '
          f'expected gas: {insert_gas["gas_expected"]}, '
          f'actual gas: {insert_gas["gas_actual"]}')
    clear_output(wait=True) # Clear terminal output.

Actual gas values are now more closely approximated by pure storage gas costs:

In [ ]:
df3 = pd.DataFrame(gas)
ax3 = df3.plot(x='n_keys',
               y='gas_expected')
df3.plot(x='n_keys',
         y='gas_actual',
         grid=True,
         ax=ax3);
ax3.set_ylim(ymin=0);

Here, each time a new bit gets set, e.g. after `0b00011111` has been inserted and then `0b00100000` gets inserted, insertion requires walking down the tree and then back up to the root to insert above.
Hence the 7 spikes in gas costs at regular intervals.

Borrowing is a single read operation:

In [ ]:
expected_read_cost(128)

In [ ]:
tx_diagnostics(get_tx_json(borrow(1)))

Removing is a single write operation:

In [ ]:
expected_write_cost(128)

In [ ]:
tx_diagnostics(get_tx_json(pop(1)))

Clearing out all elements is also a single write operation, but requires extensive iteration:

In [ ]:
expected_write_cost(127)

In [ ]:
tx_diagnostics(get_tx_json(clear()))

# Access multiplicity

First insert 3 keys:

In [ ]:
# Reset tree store, waiting until tx clears.
tx_diagnostics(get_tx_json(reset()))
tx_diagnostics(get_tx_json(insert(1)))
tx_diagnostics(get_tx_json(insert(2)))
tx_diagnostics(get_tx_json(insert(3)))

Here, the `pop` function borrows the `TreeStore`, then operates on the `CritBitTree`, for a single per-item write:

In [ ]:
expected_write_cost(3)

In [ ]:
tx_diagnostics(get_tx_json(pop(3)))

The `pop_twice` function repeates the process twice in a single transaction, returning in between: it reads the `TreeStore`, then writes to the `CritBitTree`, then returns, then reads the `TreeStore`, then again writes to the `CritBitTree`.
Yet because the same item in global memory is written to each time, the transaction is only assessed a single per-item write:

In [ ]:
expected_write_cost(2)

In [ ]:
tx_diagnostics(get_tx_json(pop_twice(1, 2)))